# Module 06


In [1]:
import random
from collections import Counter
from pathlib import Path

import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import polars.selectors as cs
from IPython.display import display
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import (
    accuracy_score,
    auc,
    mutual_info_score,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import KFold, train_test_split
from sklearn.tree import DecisionTreeClassifier, export_text
from tqdm.auto import tqdm

from ml_zoomcamp.utils import clean_column_names, load_data, normalize_name

alt.data_transformers.disable_max_rows()

ROOT_DIR = Path.cwd().parent
DATA_DIR = ROOT_DIR.joinpath("data")

## 1. Data Preparation


In [2]:
csv_uri = "https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-06-trees/CreditScoring.csv"

df = load_data(csv_uri, DATA_DIR, null_values=["99999999"])
df = clean_column_names(df)

In [3]:
df.glimpse()

Rows: 4455
Columns: 14
$ status    <i64> 1, 1, 2, 1, 1, 1, 1, 1, 1, 2
$ seniority <i64> 9, 17, 10, 0, 0, 1, 29, 9, 0, 0
$ home      <i64> 1, 1, 2, 1, 1, 2, 2, 5, 2, 5
$ time      <i64> 60, 60, 36, 60, 36, 60, 60, 12, 60, 48
$ age       <i64> 30, 58, 46, 24, 26, 36, 44, 27, 32, 41
$ marital   <i64> 2, 3, 2, 1, 1, 2, 2, 1, 2, 2
$ records   <i64> 1, 1, 2, 1, 1, 1, 1, 1, 1, 1
$ job       <i64> 3, 1, 3, 1, 1, 1, 1, 1, 3, 2
$ expenses  <i64> 73, 48, 90, 63, 46, 75, 75, 35, 90, 90
$ income    <i64> 129, 131, 200, 182, 107, 214, 125, 80, 107, 80
$ assets    <i64> 0, 0, 3000, 2500, 0, 3500, 10000, 0, 15000, 0
$ debt      <i64> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
$ amount    <i64> 800, 1000, 2000, 900, 310, 650, 1600, 200, 1200, 1200
$ price     <i64> 846, 1658, 2985, 1325, 910, 1645, 1800, 1093, 1957, 1468



In [4]:
status_values = ["unk", "ok", "default"]
home_values = ["unk", "rent", "owner", "private", "ignore", "parents", "other"]
marital_values = ["unk", "single", "married", "widow", "separated", "divorced"]
records_values = ["unk", "no", "yes"]
job_values = ["unk", "fixed", "partime", "freelance", "others"]

status_dtype = pl.Enum(status_values)
home_dtype = pl.Enum(home_values)
marital_dtype = pl.Enum(marital_values)
records_dtype = pl.Enum(records_values)
job_dtype = pl.Enum(job_values)

In [5]:
df = df.with_columns(
    pl.col("status").replace_strict(
        range(len(status_values)), status_values, return_dtype=status_dtype
    ),
    pl.col("home").replace_strict(
        range(len(home_values)), home_values, return_dtype=home_dtype
    ),
    pl.col("marital").replace_strict(
        range(len(marital_values)), marital_values, return_dtype=marital_dtype
    ),
    pl.col("records").replace_strict(
        range(len(records_values)), records_values, return_dtype=records_dtype
    ),
    pl.col("job").replace_strict(
        range(len(job_values)), job_values, return_dtype=job_dtype
    ),
)

In [6]:
df.head()

status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
enum,i64,enum,i64,i64,enum,enum,enum,i64,i64,i64,i64,i64,i64
"""ok""",9,"""rent""",60,30,"""married""","""no""","""freelance""",73,129,0,0,800,846
"""ok""",17,"""rent""",60,58,"""widow""","""no""","""fixed""",48,131,0,0,1000,1658
"""default""",10,"""owner""",36,46,"""married""","""yes""","""freelance""",90,200,3000,0,2000,2985
"""ok""",0,"""rent""",60,24,"""single""","""no""","""fixed""",63,182,2500,0,900,1325
"""ok""",0,"""rent""",36,26,"""single""","""no""","""fixed""",46,107,0,0,310,910


In [7]:
df.describe()

statistic,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
str,str,f64,str,f64,f64,str,str,str,f64,f64,f64,f64,f64,f64
"""count""","""4455""",4455.0,"""4455""",4455.0,4455.0,"""4455""","""4455""","""4455""",4455.0,4421.0,4408.0,4437.0,4455.0,4455.0
"""null_count""","""0""",0.0,"""0""",0.0,0.0,"""0""","""0""","""0""",0.0,34.0,47.0,18.0,0.0,0.0
"""mean""",null,7.987205,null,46.441751,37.077666,null,null,null,55.568799,130.568197,5403.433984,342.948614,1039.021773,1462.875645
"""std""",null,8.173444,null,14.655225,10.984856,null,null,null,19.515878,86.367434,11573.161523,1245.861736,474.543007,628.089913
"""min""",null,0.0,null,6.0,18.0,null,null,null,35.0,0.0,0.0,0.0,100.0,105.0
"""25%""",null,2.0,null,36.0,28.0,null,null,null,35.0,80.0,0.0,0.0,700.0,1118.0
"""50%""",null,5.0,null,48.0,36.0,null,null,null,51.0,120.0,3000.0,0.0,1000.0,1400.0
"""75%""",null,12.0,null,60.0,45.0,null,null,null,72.0,165.0,6000.0,0.0,1300.0,1692.0
"""max""",null,48.0,null,72.0,68.0,null,null,null,180.0,959.0,300000.0,30000.0,5000.0,11140.0


In [8]:
df = df.with_columns(pl.col("income", "assets", "debt").replace(99999999, None))
df.describe()

statistic,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
str,str,f64,str,f64,f64,str,str,str,f64,f64,f64,f64,f64,f64
"""count""","""4455""",4455.0,"""4455""",4455.0,4455.0,"""4455""","""4455""","""4455""",4455.0,4421.0,4408.0,4437.0,4455.0,4455.0
"""null_count""","""0""",0.0,"""0""",0.0,0.0,"""0""","""0""","""0""",0.0,34.0,47.0,18.0,0.0,0.0
"""mean""",null,7.987205,null,46.441751,37.077666,null,null,null,55.568799,130.568197,5403.433984,342.948614,1039.021773,1462.875645
"""std""",null,8.173444,null,14.655225,10.984856,null,null,null,19.515878,86.367434,11573.161523,1245.861736,474.543007,628.089913
"""min""",null,0.0,null,6.0,18.0,null,null,null,35.0,0.0,0.0,0.0,100.0,105.0
"""25%""",null,2.0,null,36.0,28.0,null,null,null,35.0,80.0,0.0,0.0,700.0,1118.0
"""50%""",null,5.0,null,48.0,36.0,null,null,null,51.0,120.0,3000.0,0.0,1000.0,1400.0
"""75%""",null,12.0,null,60.0,45.0,null,null,null,72.0,165.0,6000.0,0.0,1300.0,1692.0
"""max""",null,48.0,null,72.0,68.0,null,null,null,180.0,959.0,300000.0,30000.0,5000.0,11140.0


In [9]:
df.null_count().transpose(include_header=True, column_names=["null_count"]).filter(
    pl.col("null_count") > 0
).sort(pl.col("null_count"), descending=True)

column,null_count
str,u32
"""assets""",47
"""income""",34
"""debt""",18


In [10]:
df["status"].value_counts()

status,count
enum,u32
"""default""",1254
"""ok""",3200
"""unk""",1


In [11]:
df = df.filter(pl.col("status") != "unk")

In [12]:
df.head()

status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
enum,i64,enum,i64,i64,enum,enum,enum,i64,i64,i64,i64,i64,i64
"""ok""",9,"""rent""",60,30,"""married""","""no""","""freelance""",73,129,0,0,800,846
"""ok""",17,"""rent""",60,58,"""widow""","""no""","""fixed""",48,131,0,0,1000,1658
"""default""",10,"""owner""",36,46,"""married""","""yes""","""freelance""",90,200,3000,0,2000,2985
"""ok""",0,"""rent""",60,24,"""single""","""no""","""fixed""",63,182,2500,0,900,1325
"""ok""",0,"""rent""",36,26,"""single""","""no""","""fixed""",46,107,0,0,310,910


## 2. Setting Up Validation Framework


In [13]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=11)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=11)

In [14]:
len(df_train), len(df_val), len(df_test)

(2672, 891, 891)

In [15]:
y_train = (df_train["status"] == "default").cast(pl.Int8).to_numpy()
y_val = (df_val["status"] == "default").cast(pl.Int8).to_numpy()
y_test = (df_test["status"] == "default").cast(pl.Int8).to_numpy()

In [16]:
df_train = df_train.drop("status")
df_val = df_val.drop("status")
df_test = df_test.drop("status")

## 3. Exploratory Data Analysis


In [17]:
df_full_train.null_count().transpose(
    include_header=True, column_names=["null_count"]
).filter(pl.col("null_count") > 0)

column,null_count
str,u32
"""income""",30
"""assets""",38
"""debt""",14


In [18]:
df_full_train["status"].value_counts(sort=True, normalize=True)

status,proportion
enum,f64
"""ok""",0.720741
"""default""",0.279259


## 4. Encoding


In [19]:
train_dicts = df_train.fill_null(0).to_dicts()

In [20]:
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)

In [21]:
dv.feature_names_[:10]

['age',
 'amount',
 'assets',
 'debt',
 'expenses',
 'home=ignore',
 'home=other',
 'home=owner',
 'home=parents',
 'home=private']

## 5. Decision Trees


In [22]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

DecisionTreeClassifier()

In [23]:
val_dicts = df_val.fill_null(0).to_dicts()
X_val = dv.transform(val_dicts)

In [24]:
y_pred = dt.predict_proba(X_val)[:, 1]

In [25]:
roc_auc_score(y_val, y_pred)

np.float64(0.6575373568089897)

In [26]:
y_pred = dt.predict_proba(X_train)[:, 1]
roc_auc_score(y_train, y_pred)

np.float64(1.0)

Resulted in overfitting


### Retrain to handle overfitting


In [27]:
dt = DecisionTreeClassifier(max_depth=2)
dt.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=2)

In [28]:
y_pred = dt.predict_proba(X_train)[:, 1]
train_auc = roc_auc_score(y_train, y_pred)
print("train:", train_auc)

y_pred = dt.predict_proba(X_val)[:, 1]
val_auc = roc_auc_score(y_val, y_pred)
print("val:", val_auc)


train: 0.7054989859726213
val: 0.6685264343319367


In [29]:
print(export_text(dt, feature_names=dv.feature_names_))

|--- records=yes <= 0.50
|   |--- job=partime <= 0.50
|   |   |--- class: 0
|   |--- job=partime >  0.50
|   |   |--- class: 1
|--- records=yes >  0.50
|   |--- seniority <= 6.50
|   |   |--- class: 1
|   |--- seniority >  6.50
|   |   |--- class: 0



## 6. Decision Tree Learning Algorithm

Using misclassification rate


### One Feature


In [30]:
data = [
    [8000, "default"],
    [2000, "default"],
    [0, "default"],
    [5000, "ok"],
    [5000, "ok"],
    [4000, "ok"],
    [9000, "ok"],
    [3000, "default"],
]
df_example = pl.DataFrame(data, schema=["assets", "status"], orient="row")
df_example

assets,status
i64,str
8000,"""default"""
2000,"""default"""
0,"""default"""
5000,"""ok"""
5000,"""ok"""
4000,"""ok"""
9000,"""ok"""
3000,"""default"""


In [31]:
df_example.sort("assets")

assets,status
i64,str
0,"""default"""
2000,"""default"""
3000,"""default"""
4000,"""ok"""
5000,"""ok"""
5000,"""ok"""
8000,"""default"""
9000,"""ok"""


In [32]:
Ts = [0, 2000, 3000, 4000, 5000, 8000]

In [33]:
T = 4000
df_left = df_example.filter(pl.col("assets") <= T)
df_right = df_example.filter(pl.col("assets") > T)

display(df_left)
print(df_left["status"].value_counts(normalize=True))
display(df_right)
print(df_right["status"].value_counts(normalize=True))

assets,status
i64,str
2000,"""default"""
0,"""default"""
4000,"""ok"""
3000,"""default"""


shape: (2, 2)
┌─────────┬────────────┐
│ status  ┆ proportion │
│ ---     ┆ ---        │
│ str     ┆ f64        │
╞═════════╪════════════╡
│ ok      ┆ 0.25       │
│ default ┆ 0.75       │
└─────────┴────────────┘


assets,status
i64,str
8000,"""default"""
5000,"""ok"""
5000,"""ok"""
9000,"""ok"""


shape: (2, 2)
┌─────────┬────────────┐
│ status  ┆ proportion │
│ ---     ┆ ---        │
│ str     ┆ f64        │
╞═════════╪════════════╡
│ default ┆ 0.25       │
│ ok      ┆ 0.75       │
└─────────┴────────────┘


In [34]:
for T in Ts:
    print(T)
    df_left = df_example.filter(pl.col("assets") <= T)
    df_right = df_example.filter(pl.col("assets") > T)

    display(df_left)
    print(df_left["status"].value_counts(normalize=True))
    display(df_right)
    print(df_right["status"].value_counts(normalize=True))

    print("-" * 40)

0


assets,status
i64,str
0,"""default"""


shape: (1, 2)
┌─────────┬────────────┐
│ status  ┆ proportion │
│ ---     ┆ ---        │
│ str     ┆ f64        │
╞═════════╪════════════╡
│ default ┆ 1.0        │
└─────────┴────────────┘


assets,status
i64,str
8000,"""default"""
2000,"""default"""
5000,"""ok"""
5000,"""ok"""
4000,"""ok"""
9000,"""ok"""
3000,"""default"""


shape: (2, 2)
┌─────────┬────────────┐
│ status  ┆ proportion │
│ ---     ┆ ---        │
│ str     ┆ f64        │
╞═════════╪════════════╡
│ ok      ┆ 0.571429   │
│ default ┆ 0.428571   │
└─────────┴────────────┘
----------------------------------------
2000


...

shape: (2, 2)
┌─────────┬────────────┐
│ status  ┆ proportion │
│ ---     ┆ ---        │
│ str     ┆ f64        │
╞═════════╪════════════╡
│ default ┆ 0.5        │
│ ok      ┆ 0.5        │
└─────────┴────────────┘
----------------------------------------
8000


assets,status
i64,str
8000,"""default"""
2000,"""default"""
0,"""default"""
5000,"""ok"""
5000,"""ok"""
4000,"""ok"""
3000,"""default"""


shape: (2, 2)
┌─────────┬────────────┐
│ status  ┆ proportion │
│ ---     ┆ ---        │
│ str     ┆ f64        │
╞═════════╪════════════╡
│ default ┆ 0.571429   │
│ ok      ┆ 0.428571   │
└─────────┴────────────┘


assets,status
i64,str
9000,"""ok"""


shape: (1, 2)
┌────────┬────────────┐
│ status ┆ proportion │
│ ---    ┆ ---        │
│ str    ┆ f64        │
╞════════╪════════════╡
│ ok     ┆ 1.0        │
└────────┴────────────┘
----------------------------------------


### Two Features


In [35]:
data = [
    [8000, 3000, "default"],
    [2000, 1000, "default"],
    [0, 1000, "default"],
    [5000, 1000, "ok"],
    [5000, 1000, "ok"],
    [4000, 1000, "ok"],
    [9000, 500, "ok"],
    [3000, 2000, "default"],
]
df_example = pl.DataFrame(data, schema=["assets", "debt", "status"], orient="row")
df_example

assets,debt,status
i64,i64,str
8000,3000,"""default"""
2000,1000,"""default"""
0,1000,"""default"""
5000,1000,"""ok"""
5000,1000,"""ok"""
4000,1000,"""ok"""
9000,500,"""ok"""
3000,2000,"""default"""


In [36]:
df_example.sort("debt")

assets,debt,status
i64,i64,str
9000,500,"""ok"""
2000,1000,"""default"""
0,1000,"""default"""
5000,1000,"""ok"""
5000,1000,"""ok"""
4000,1000,"""ok"""
3000,2000,"""default"""
8000,3000,"""default"""


In [37]:
thresholds = {"assets": [0, 2000, 3000, 4000, 5000, 8000], "debt": [500, 1000, 2000]}

In [38]:
next(
    iter(
        df_left["status"]
        .value_counts(normalize=True)
        .filter(pl.col("status") == "default")["proportion"]
    ),
    None,
)

0.5714285714285714

In [39]:
df_left["status"].value_counts(normalize=True).filter(pl.col("status") == "default")[
    "proportion"
]

proportion
f64
0.571429


In [40]:
impurities = []
for feature, Ts in thresholds.items():
    print(feature)
    for T in Ts:
        print(T)
        df_left = df_example.filter(pl.col(feature) <= T)
        df_right = df_example.filter(pl.col(feature) > T)

        display(df_left)
        print(df_left["status"].value_counts(normalize=True))
        display(df_right)
        print(df_right["status"].value_counts(normalize=True))

        decision_left = "default" if feature == "assets" else "ok"
        decision_right = "ok" if feature == "assets" else "default"

        impurity_left = next(
            iter(
                df_left["status"]
                .value_counts(normalize=True)
                .filter(pl.col("status") == decision_right)["proportion"]
            ),
            0,
        )
        impurity_right = next(
            iter(
                df_right["status"]
                .value_counts(normalize=True)
                .filter(pl.col("status") == decision_left)["proportion"]
            ),
            0,
        )

        print("-" * 40)
        impurities.append(
            (feature, T, decision_left, impurity_left, decision_right, impurity_right)
        )
    print("#" * 40)

assets
0


assets,debt,status
i64,i64,str
0,1000,"""default"""


shape: (1, 2)
┌─────────┬────────────┐
│ status  ┆ proportion │
│ ---     ┆ ---        │
│ str     ┆ f64        │
╞═════════╪════════════╡
│ default ┆ 1.0        │
└─────────┴────────────┘


assets,debt,status
i64,i64,str
8000,3000,"""default"""
2000,1000,"""default"""
5000,1000,"""ok"""
5000,1000,"""ok"""
4000,1000,"""ok"""
9000,500,"""ok"""
3000,2000,"""default"""


shape: (2, 2)
┌─────────┬────────────┐
│ status  ┆ proportion │
│ ---     ┆ ---        │
│ str     ┆ f64        │
╞═════════╪════════════╡
│ ok      ┆ 0.571429   │
│ default ┆ 0.428571   │
└─────────┴────────────┘
----------------------------------------
2000


...

shape: (1, 2)
┌─────────┬────────────┐
│ status  ┆ proportion │
│ ---     ┆ ---        │
│ str     ┆ f64        │
╞═════════╪════════════╡
│ default ┆ 1.0        │
└─────────┴────────────┘
----------------------------------------
2000


assets,debt,status
i64,i64,str
2000,1000,"""default"""
0,1000,"""default"""
5000,1000,"""ok"""
5000,1000,"""ok"""
4000,1000,"""ok"""
9000,500,"""ok"""
3000,2000,"""default"""


shape: (2, 2)
┌─────────┬────────────┐
│ status  ┆ proportion │
│ ---     ┆ ---        │
│ str     ┆ f64        │
╞═════════╪════════════╡
│ ok      ┆ 0.571429   │
│ default ┆ 0.428571   │
└─────────┴────────────┘


assets,debt,status
i64,i64,str
8000,3000,"""default"""


shape: (1, 2)
┌─────────┬────────────┐
│ status  ┆ proportion │
│ ---     ┆ ---        │
│ str     ┆ f64        │
╞═════════╪════════════╡
│ default ┆ 1.0        │
└─────────┴────────────┘
----------------------------------------
########################################


In [41]:
impurities_columns = (
    "feature",
    "t",
    "decision_left",
    "impurity_left",
    "decision_right",
    "impurity_right",
)
df_impurities = pl.DataFrame(
    impurities,
    schema=impurities_columns,
    orient="row",
).with_columns(impurity_mean=pl.mean_horizontal(cs.contains("impurity")))
display(df_impurities)
display(
    df_impurities.sort("impurity_mean")
    .group_by("feature")
    .agg(pl.col("t").first(), pl.col("impurity_mean").min())
)

feature,t,decision_left,impurity_left,decision_right,impurity_right,impurity_mean
str,i64,str,f64,str,f64,f64
"""assets""",0,"""default""",0.0,"""ok""",0.428571,0.214286
"""assets""",2000,"""default""",0.0,"""ok""",0.333333,0.166667
"""assets""",3000,"""default""",0.0,"""ok""",0.2,0.1
"""assets""",4000,"""default""",0.25,"""ok""",0.25,0.25
"""assets""",5000,"""default""",0.5,"""ok""",0.5,0.5
"""assets""",8000,"""default""",0.428571,"""ok""",0.0,0.214286
"""debt""",500,"""ok""",0.0,"""default""",0.428571,0.214286
"""debt""",1000,"""ok""",0.333333,"""default""",0.0,0.166667
"""debt""",2000,"""ok""",0.428571,"""default""",0.0,0.214286


feature,t,impurity_mean
str,i64,f64
"""assets""",3000,0.1
"""debt""",1000,0.166667


### Stopping criteria

- Group already pure
- Tree reached depth limit
- Group too small to split


## 7. Decision Tree Parameter Tuning

- selecting `max_depth`
- selecting `min_samples_leaf`


In [42]:
for d in [1, 2, 3, 4, 5, 6, 10, 15, 20, None]:
    dt = DecisionTreeClassifier(max_depth=d)
    dt.fit(X_train, y_train)

    y_pred = dt.predict_proba(X_val)[:, 1]
    val_auc = roc_auc_score(y_val, y_pred)

    print("%4s -> %.3f" % (d, val_auc))
    # print("{:>4} -> {:.3f}".format(str(d), val_auc))
    # print(f"{str(d):>4} -> {val_auc:.3f}")
    # print(f"{format(str(d),">4")} -> {format(val_auc,".3f")}")

   1 -> 0.606
   2 -> 0.669
   3 -> 0.739
   4 -> 0.761
   5 -> 0.766
   6 -> 0.759
  10 -> 0.691
  15 -> 0.678
  20 -> 0.664
None -> 0.651


In [43]:
scores = []
for d in [4, 5, 6, 7, 10, 15, 20, None]:
    for s in [1, 2, 5, 10, 15, 20, 100, 200, 500]:
        dt = DecisionTreeClassifier(max_depth=d, min_samples_leaf=s)
        dt.fit(X_train, y_train)

        y_pred = dt.predict_proba(X_val)[:, 1]
        val_auc = roc_auc_score(y_val, y_pred)

        scores.append((d, s, val_auc))

In [44]:
scores_columns = ("max_depth", "min_samples_leaf", "auc")
df_scores = pl.DataFrame(scores, schema=scores_columns, orient="row")
df_scores.head()

max_depth,min_samples_leaf,auc
i64,i64,f64
4,1,0.761283
4,2,0.761283
4,5,0.761283
4,10,0.761283
4,15,0.763726


In [45]:
df_scores.sort(pl.col("auc"), descending=True).head()

max_depth,min_samples_leaf,auc
i64,i64,f64
10,15,0.791495
20,15,0.787829
null,15,0.78769
15,15,0.786875
6,15,0.785474


In [46]:
df_scores_pivot = df_scores.pivot(
    on="max_depth", index="min_samples_leaf", values="auc"
)
df_scores_pivot.select(cs.all().round(3))

min_samples_leaf,4,5,6,7,10,15,20,null
i64,f64,f64,f64,f64,f64,f64,f64,f64
1,0.761,0.766,0.746,0.748,0.696,0.651,0.655,0.653
2,0.761,0.766,0.755,0.75,0.725,0.694,0.69,0.685
5,0.761,0.768,0.762,0.758,0.735,0.715,0.715,0.716
10,0.761,0.762,0.778,0.763,0.765,0.76,0.758,0.761
15,0.764,0.772,0.785,0.78,0.791,0.787,0.788,0.788
20,0.761,0.774,0.774,0.781,0.785,0.782,0.781,0.782
100,0.756,0.763,0.776,0.78,0.779,0.78,0.78,0.78
200,0.747,0.759,0.768,0.768,0.768,0.768,0.768,0.768
500,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68


In [47]:
heatmap = (
    alt.Chart(df_scores)
    .mark_rect()
    .encode(
        alt.X("max_depth:O", title="Max Depth"),
        alt.Y("min_samples_leaf:O", title="Min Samples Leaf"),
        alt.Color("auc:Q", title="AUC", scale=alt.Scale(scheme="viridis")),
    )
    .properties(width=300, height=300)
)

text = heatmap.mark_text().encode(
    text=alt.Text("auc:Q", format=".3f"),
    color=alt.condition(
        alt.datum.auc > 0.73,
        alt.value("black"),
        alt.value("white"),
    ),
)

chart = heatmap + text
chart

alt.LayerChart(...)

In [48]:
dt = DecisionTreeClassifier(max_depth=6, min_samples_leaf=15)
dt.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=15)

## 8. Ensemble Learning and Random Forest
